<a href="https://colab.research.google.com/github/krishikasahni/Deep-Learning/blob/main/Deep_Learning_Assignment_7_Sequence_to_Sequence_Learning_with_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

data_path = "/kaggle/input/datasets/krishikasahni/spatxt/spa.txt"
lines = open(data_path, encoding="utf-8").read().split("\n")[:20000]

input_texts = []
target_texts = []

for line in lines:
    if "\t" in line:
        eng, spa = line.split("\t")[:2]
        input_texts.append(eng)
        target_texts.append("\t " + spa + " \n")

num_words = 5000

input_tok = Tokenizer(num_words=num_words, filters='')
input_tok.fit_on_texts(input_texts)
input_seq = input_tok.texts_to_sequences(input_texts)

target_tok = Tokenizer(num_words=num_words, filters='')
target_tok.fit_on_texts(target_texts)
target_seq = target_tok.texts_to_sequences(target_texts)

max_enc = max(len(s) for s in input_seq)
max_dec = max(len(s) for s in target_seq)

encoder_input = pad_sequences(input_seq, maxlen=max_enc, padding='post')
decoder_input = pad_sequences(target_seq, maxlen=max_dec, padding='post')

decoder_target = np.zeros_like(decoder_input)
decoder_target[:, :-1] = decoder_input[:, 1:]

latent_dim = 128

enc_inputs = Input(shape=(None,))
enc_emb = Embedding(num_words, latent_dim)(enc_inputs)
_, state_h, state_c = LSTM(latent_dim, return_state=True)(enc_emb)
enc_states = [state_h, state_c]

dec_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_words, latent_dim)
dec_emb = dec_emb_layer(dec_inputs)

dec_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
dec_outputs, _, _ = dec_lstm(dec_emb, initial_state=enc_states)
dec_dense = Dense(num_words, activation='softmax')
dec_outputs = dec_dense(dec_outputs)

model = Model([enc_inputs, dec_inputs], dec_outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(
    [encoder_input, decoder_input],
    np.expand_dims(decoder_target, -1),
    batch_size=128,
    epochs=5,
    validation_split=0.2
)

encoder_model = Model(enc_inputs, enc_states)

dec_state_input_h = Input(shape=(latent_dim,))
dec_state_input_c = Input(shape=(latent_dim,))
dec_states_inputs = [dec_state_input_h, dec_state_input_c]

dec_emb2 = dec_emb_layer(dec_inputs)
dec_outputs2, state_h2, state_c2 = dec_lstm(dec_emb2, initial_state=dec_states_inputs)
dec_states2 = [state_h2, state_c2]
dec_outputs2 = dec_dense(dec_outputs2)

decoder_model = Model(
    [dec_inputs] + dec_states_inputs,
    [dec_outputs2] + dec_states2
)

def translate_sentence(sentence):
    seq = input_tok.texts_to_sequences([sentence])
    seq = pad_sequences(seq, maxlen=max_enc, padding='post')

    states = encoder_model.predict(seq, verbose=0)

    target_seq = np.zeros((1,1))
    target_seq[0,0] = target_tok.word_index['\t']

    decoded_sentence = ''
    while True:
        output_tokens, h, c = decoder_model.predict([target_seq] + states, verbose=0)
        idx = np.argmax(output_tokens[0, -1, :])
        word = target_tok.index_word.get(idx, '')

        if word == '\n' or len(decoded_sentence.split()) > 20:
            break

        decoded_sentence += ' ' + word
        target_seq[0,0] = idx
        states = [h, c]

    return decoded_sentence.strip()

while True:
    user_input = input("\nEnter English sentence (or type 'exit'): ")
    if user_input.lower() == "exit":
        break
    print("Spanish:", translate_sentence(user_input))

Epoch 1/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 6s 30ms/step - accuracy: 0.6726 - loss: 4.5759 - val_accuracy: 0.6589 - val_loss: 2.0742
Epoch 2/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - accuracy: 0.7526 - loss: 1.6588 - val_accuracy: 0.7356 - val_loss: 1.8884
Epoch 3/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - accuracy: 0.7855 - loss: 1.5167 - val_accuracy: 0.7401 - val_loss: 1.8050
Epoch 4/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - accuracy: 0.7891 - loss: 1.4411 - val_accuracy: 0.7415 - val_loss: 1.7497
Epoch 5/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - accuracy: 0.7916 - loss: 1.3757 - val_accuracy: 0.7485 - val_loss: 1.6943



Enter English sentence (or type 'exit'):  How are you?


Spanish: yo me



Enter English sentence (or type 'exit'):  I am good


Spanish: me a



Enter English sentence (or type 'exit'):  exit
